In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import csv
import datetime
import chromedriver_autoinstaller

# 設置 Chrome 瀏覽器的選項
chrome_options = Options()
chrome_options.add_argument('--headless')  # 隱藏瀏覽器窗口
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')

# 自動安裝 ChromeDriver
chromedriver_autoinstaller.install()

# 初始化 WebDriver
driver = webdriver.Chrome(options=chrome_options)

# 訪問網站
url = "https://www.cmoney.tw/finance/2603/f00038"
driver.get(url)

try:
    # 等待所有表格元素加載出來
    wait = WebDriverWait(driver, 10)
    tables = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "tb1")))
    
    # 選擇第二個表格
    if len(tables) >= 2:
        table = tables[1]
    else:
        raise Exception("Failed to find the second target table.")
    
    # 找到所有行元素
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    headers = []
    for idx, row in enumerate(rows):
        if idx == 0:  # 第一行是表格的標題
            headers = [col.text.strip() for col in row.find_elements(By.TAG_NAME, "th")]
        else:
            columns = row.find_elements(By.TAG_NAME, "td")
            row_data = [col.text.strip() for col in columns]
            if row_data:
                data.append(row_data)
    
    # 如果表格有表頭信息，將其與表格數據合併
    if headers:
        # 獲取當前日期並計算是第幾週
        today = datetime.date.today()
        year, week_num, _ = today.isocalendar()
        week_str = f'w{week_num:02}'
        
        # 將數據寫入 CSV 文件，文件名包含年份和第幾週
        filename = f'2603_share_distribution_{year}_{week_str}.csv'
        with open(filename, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(headers)
            writer.writerows(data)
        
        print(f"Data has been written to {filename}")
        
        # 印出第一列的第 3, 4, 5 欄資料
        if len(data) > 0 and len(data[0]) >= 5:
            print(f"兩個月前大於400張: {data[1][-3]}%")
            print(f"上個月大於400張: {data[1][-2]}%")
            print(f"本月大於400張: {data[1][-1]}%")
        else:
            print("Not enough columns in the first row to print 3rd, 4th, and 5th values.")
    else:
        print("Failed to find headers in the table.")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # 關閉瀏覽器
    driver.quit()

# 注意：
# 這段程式碼使用 Selenium 來模擬瀏覽器行為，因此可以抓取動態加載的數據。請確保您已安裝 ChromeDriver，並根據您的環境設置 PATH。

Data has been written to 2603_share_distribution_2024_w45.csv
兩個月前大於400張: 69.07%
上個月大於400張: 69.16%
本月大於400張: 69.38%
